# CNN

In [ ]:
import os
import numpy as np
import pickle
import keras
from matplotlib import pyplot as plt

from keras.models import Sequential, Input, Model
from keras.layers import Dense, Dropout, Flatten, LSTM, Embedding, Activation
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras.regularizers import l2 # L2-regularisation
from keras.activations import sigmoid
from keras.models import Model
import tensorflow as tf
import warnings

### Data loader

This notebook has been adapted for Kaggle platform

In [ ]:
def load_data(data_name):
    pkl_file = open(data_name + '.pkl', 'rb')
    data = pickle.load(pkl_file)
    pkl_file.close()
    return data

print('Loading data...')
folder = '../input/neuraldataset2540/'

train = load_data(folder + 'neural_train')
train_label = load_data(folder + 'neural_train_label')
test = load_data(folder + 'neural_test')
test_label = load_data(folder + 'neural_test_label')
valid = load_data(folder + 'neural_valid')
valid_label = load_data(folder + 'neural_valid_label')
print('Data are loaded')


In [ ]:
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

warnings.filterwarnings("ignore")

l2_lambda = 0.0001
batch_size = 8
epochs = 25
reg = l2(l2_lambda)
init="he_normal"
mm1 = 40
mm = 40

### CNN

In [ ]:
def cnn(input_shape, mm, output_shape):

    input_layer = Input(input_shape)

    layer3 = (Conv2D(32, (mm, 5), activation='linear', W_regularizer=l2(l2_lambda), padding='same'))(input_layer)
    layer3 = (LeakyReLU(alpha=0.1))(layer3)
    layer3 = (Conv2D(64, (mm, 5), activation='linear', W_regularizer=l2(l2_lambda), padding='same'))(layer3)
    layer3 = (LeakyReLU(alpha=0.1))(layer3)
    layer3 = (MaxPooling2D(pool_size=(2, 2),padding='same'))(layer3)
    layer3 = (Dropout(0.3))(layer3)

    layer4 = (Flatten())(layer3)
    layer4 = (Dense((mm), activation='linear', W_regularizer=l2(l2_lambda)))(layer4)
    layer4 = (LeakyReLU(alpha=0.1))(layer4)
    layer4 = (Dropout(0.3))(layer4)
    finish = []

    for i in range(int((mm ** 2 - mm) / 2)):
        output = Dense(output_shape, activation='softmax', name='main_output'+str(i))(layer4)
        finish.append(output)

    model = Model(inputs=[input_layer], outputs=output)
    model.compile(loss = "categorical_crossentropy", optimizer=keras.optimizers.Adam(lr=0.001))
    
    return model

Training and making predictions

In [ ]:
model = cnn((train.shape[1], train.shape[2], 1), mm, 2)

train_dropout = model.fit(train, train_label, batch_size = batch_size, epochs = epochs, verbose = 1,
                          validation_data = (np.array(valid), valid_label))

predicted = model.predict(test)

print(np.shape(predicted))
print(np.shape(test_label))

output = open('predicted2' + '.pkl', 'wb')
pickle.dump(predicted, output)
output.close()

Plot the loss

In [ ]:
# Plot training & validation loss values
plt.plot(train_dropout.history['loss'])
plt.plot(train_dropout.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()